__Get the Tweet_Ids__

In [43]:
import pandas as pd
por = pd.read_csv("C:/Users/gsent/Documents/Spring_2020/Info_440/eq_tweet_id.csv")

In [52]:
import json
from pprint import pprint
import requests
from subprocess import check_output
import time

#################################################
#  EDIT THESE VARIABLES BEFORE STARTING SCRIPT  #
#################################################

FILEPATH = "C:/Users/gsent/Documents/Spring_2020/Info_440/eq_chile_batch.csv"
CONSUMER_KEY = "9XG3noRIh1aHsumCnHpyyLvzg"
CONSUMER_SECRET = "yYu0U537ExdDbh0t525n1p5RQBLdznP5NIN3YOufd5h5KcRFPv"

def get_twitter_oauth_token(key=CONSUMER_KEY, secret=CONSUMER_SECRET):
    """ Generate a new authorization token from the Twitter API using curl subprocesses.
    """
    # See https://developer.twitter.com/en/docs/basics/authentication/overview/oauth
    # for more information on Twitter's OAuth implementation.
    
    args = [
        'curl',
        '-u',
        '{0}:{1}'.format(key, secret),
        '--data',
        'grant_type=client_credentials',
        'https://api.twitter.com/oauth2/token'
    ]
    
    # Run the command
    output = check_output(args)

    # Decode the byte literal output into valid JSON
    jsonRaw = output.decode('utf8').replace("'", '"')
    data = json.loads(jsonRaw)

    if (data):
        # We only care about the access_token because
        # # we know the string format of a 'Bearer' type key
        return data["access_token"]
    
    # Fail-safe
    return None


In [53]:
def read_tweet_ids_from_file(filepath=FILEPATH):
    """ Read a list of newline-delimited tweet IDS from a local file.

    :param: filepath - the location of a file on disk
    """

    tweet_ids = []
    with open(filepath, "r", encoding="utf8") as f:
        tweet_ids = [ line.strip() for line in f.readlines() ]
    
    if (len(tweet_ids) > 0):
        return tweet_ids[500:550]
    
    # Fail-safe
    return None



In [54]:
# Execute request runners
def main():
    try:
        # Request a token from the API using curl request
        token = get_twitter_oauth_token()

        # Immediately stop execution if no authorization token
        if (token is None or type(token) != str):
            print("Could not authorize, exiting.")
            exit(1)

        # Set the Authorization string
        AUTHORIZATION = "Bearer {0}".format(token)

        # Read tweets from a local file
        tweets = read_tweet_ids_from_file()
        

        # Immediately stop execution if no tweets found
        if (len(tweets) < 1):
            print("No tweets found in file, exiting.")
            exit(1)

        # Set the common request headers
        HEADERS = {
            "content-type": "application/json; charset=utf-8",
            "Authorization": AUTHORIZATION
        }

        # iterate over tweets within rate limits

        for tweet in tweets:
            # Request the tweets at found ids
            # Limit the requests to 1 per second:
            #   900 requests / 15 mins is the limit
            #   15 mins = 900 seconds, therefore
            #   1 request per 1 second
            time.sleep(1)
        
            req = requests.get("https://api.twitter.com/1.1/statuses/show/{0}.json".format(tweet), headers=HEADERS)
            
            
            if (req.status_code == 200):
                record = json.loads(req.text)
                if (record):
                    print(record)
    except Exception as error:
        print(error)
        exit(1)

# if __name__ == "__main__":
#     main()

In [55]:
main()

{'created_at': 'Wed Apr 02 02:42:10 +0000 2014', 'id': 451187806392832000, 'id_str': '451187806392832000', 'text': '#AHORA #CHILE Se despliega el ejército en las zonas afectadas tras el #sismo de 8,2 grados.', 'truncated': False, 'entities': {'hashtags': [{'text': 'AHORA', 'indices': [0, 6]}, {'text': 'CHILE', 'indices': [7, 13]}, {'text': 'sismo', 'indices': [70, 76]}], 'symbols': [], 'user_mentions': [], 'urls': []}, 'source': '<a href="http://twitter.com/#!/download/ipad" rel="nofollow">Twitter for iPad</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 253715940, 'id_str': '253715940', 'name': 'C®istina', 'screen_name': 'CriSimplemente', 'location': 'Bs.As. Argentina', 'description': 'Nunca estoy sola, vivo acompañada de mi propia soledad.', 'url': None, 'entities': {'description': {'urls': []}}, 'protected': False, 'followers_count': 28037, 'friends_co

## My Way

In [58]:
#This code creates the dataset from Corpus.csv which is downloadable from the
#internet well known dataset which is labeled manually by hand. But for the text
#of tweets you need to fetch them with their IDs.
import tweepy

# Twitter Developer keys here
# It is CENSORED
consumer_key = '9XG3noRIh1aHsumCnHpyyLvzg'
consumer_key_secret = 'yYu0U537ExdDbh0t525n1p5RQBLdznP5NIN3YOufd5h5KcRFPv'
access_token = '2758851430-YvV1cvWi8HuhY16YGAu0la6kjLbztnn8MWFn4rw'
access_token_secret = 'CyX34q3MskLwX5g5yVEyNwEJEj93GSLctvbX0qEnLwU5B'

auth = tweepy.OAuthHandler(consumer_key, consumer_key_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

# This method creates the training set
def createTrainingSet(corpusFile, targetResultFile):
    import csv
    import time

    counter = 0
    corpus = []

    with open(corpusFile, 'r') as csvfile:
        lineReader = csv.reader(csvfile, delimiter=',', quotechar="\"")
        for row in lineReader:
            corpus.append({"tweet_id": row[0]})

    sleepTime = 2
    trainingDataSet = []
    
    corpus = corpus[0:1000]

    for tweet in corpus:
        try:
            tweetFetched = api.get_status(tweet["tweet_id"])
            print("Tweet fetched" + tweetFetched.text)
            tweet["text"] = tweetFetched.text
            tweet["user"] = tweetFetched.user.name
            tweet["retweet"] = tweetFetched.retweet_count
            tweet["favorites"] = tweetFetched.favorite_count
            tweet["language"] = tweetFetched.lang
            tweet['geo'] = tweetFetched.geo
            tweet['location'] = tweetFetched.user.location
            
            
            trainingDataSet.append(tweet)
            time.sleep(sleepTime)

        except:
            print("Inside the exception - no:2")

    with open(targetResultFile, 'w') as csvfile:
        linewriter = csv.writer(csvfile, delimiter=',', quotechar="\"")
        for tweet in trainingDataSet:
            try:
                linewriter.writerow([tweet["tweet_id"], tweet["text"], tweet['user'], tweet["retweet"], tweet["favorites"], 
                                     tweet['language'],tweet['geo'], tweet['location']])
            except Exception as e:
                print(e)
    return trainingDataSet



In [59]:
# Code starts here
# This is corpus dataset
corpusFile = "C:/Users/gsent/Documents/Spring_2020/Info_440/eq_nepal_batch.csv"
# This is my target file
targetResultFile = "C:/Users/gsent/Documents/Spring_2020/Info_440/eq_tweet_actuals.csv"
# Call the method
resultFile = createTrainingSet(corpusFile, targetResultFile)

Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception

Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception

Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception - no:2
Inside the exception

## Show the resulting information that was pulled

In [19]:
resultFile

94

In [20]:
import pandas as pd

pd.DataFrame(resultFile)

,tweet_id,text,user,retweet,favorites,language,geo,location
0,503643187903488000,RT @laura_nelson: Is your home earthquake read...,Jonathan Parfrey,6,0,en,None,"Los Angeles, California"
1,503643187903488000,RT @laura_nelson: Is your home earthquake read...,Jonathan Parfrey,6,0,en,None,"Los Angeles, California"
2,503643486835712000,Real time coverage of #Napa California #earthq...,Rochester NY Buzz,0,0,en,None,"Rochester, NY"
3,503643486835712000,Real time coverage of #Napa California #earthq...,Rochester NY Buzz,0,0,en,None,"Rochester, NY"
4,503645940939776000,¦ http://t.co/xcZq0aUFSO 510 ¦ Magnitude 6.0 e...,janine,0,0,en,None,Maryland
...,...,...,...,...,...,...,...,...
89,503743098841600000,RT @WSJGraphics: Sunday's 6.0-magnitude earthq...,Nabilah Liyana,215,0,en,None,
90,503750388651008000,RT @abc7newsBayArea: First responders rushed t...,Kimberlee Sakamoto,41,0,en,None,"San Bruno, CA"
91,503750388651008000,RT @abc7newsBayArea: First responders rushed t...,Kimberlee Sakamoto,41,0,en,None,"San Bruno, CA"
92,503750831964160000,Premier Napa wineries shaken by harvest season...,Rubby Lim,0,0,en,None,Singapore


## Save the File

In [ ]:
pd.to_csv('C:/Users/gsent/Documents/Spring_2020/Info_440/eq_tweet_100k.csv')

# Read the JSON directly from CrisisNLP output

In [184]:
json_list = []

with open('C:/Users/gsent/Documents/Spring_2020/Info_440/crisisNLP/iraq_iran_earthquake_final_data.json') as f:
    for line in f:
        a = line
        b = a.replace("\n", "").strip()
        json_list.append(json.loads(b))
        


In [185]:
len(json_list)

496

In [186]:
import json
import numpy as np

att = []
#with open('C:/Users/gsent/Documents/Spring_2020/Info_440/crisisNLP/mexico_earthquake_final_data.json') as f:
for temp in json_list:
        
    text = temp['text']
    location = temp['user']['location']
    scr_name = temp['user']['screen_name']
    fol_count = temp['user']['followers_count']
    retweet = temp['retweet_count']
    favorite = temp['favorite_count']
    ver = temp['user']['verified']
        
    data = [text,location,scr_name,fol_count,retweet,favorite,ver]
    att.append(data)
    #break

In [187]:
import pandas as pd
df = pd.DataFrame(att, columns = ['text','location','screen_name','follower_count','retweets','favorites','verified'])
df

,text,location,screen_name,follower_count,retweets,favorites,verified
0,RT @PressTV: UPDATE:\nDeath toll from Iran’s q...,None,hull_arturo,45,0,0,False
1,RT @CAFOD: We pray for all those affected by t...,"Mill Hill, Blackburn",StAidanMillHill,129,0,0,False
2,RT @ReutersWorld: JUST IN: Death toll reaches ...,대한민국 인천광역시,ShayvsConnor18,256,0,0,False
3,RT @LaylaAlhussein: A magnitude 7.2 earthquake...,Pakistan,sadiqshah2112,85,0,0,False
4,Video: 7.2-magnitude earthquake jolts #Iran-Ir...,None,BehzadMoezi,6192,0,0,False
...,...,...,...,...,...,...,...
491,Damage proxy maps of 5 cities near the epicent...,None,Sadra_Krmz,3,0,0,False
492,#Iran Daily: US Treasury Blocks Donations for ...,None,EA_WorldView,3415,0,0,False
493,#Kurdish children's situation after the #earth...,Iraq,GoranShakhawan,654,0,0,False
494,#IRAN'S #EARTHQUAKE EXPOSES #POLITICAL RIFTS A...,"Washington, DC",IranNewsUpdate1,13273,0,0,True


In [188]:
#df.to_csv('C:/Users/gsent/Documents/Spring_2020/Info_440/crisisNLP/iqin_eq_data.csv', index = False)

## Create one final dataset with Classifcation of event column added

In [190]:
iriq = pd.read_csv('C:/Users/gsent/Documents/Spring_2020/Info_440/crisisNLP/iqin_eq_data.csv')
mexeq = pd.read_csv('C:/Users/gsent/Documents/Spring_2020/Info_440/crisisNLP/mexico_eq_data.csv')
calwf = pd.read_csv('C:/Users/gsent/Documents/Spring_2020/Info_440/crisisNLP/cal_fire_data.csv')
hurir = pd.read_csv('C:/Users/gsent/Documents/Spring_2020/Info_440/crisisNLP/irma_hur_data.csv')


In [191]:
len(iriq),len(mexeq),len(calwf),len(hurir)

(496, 1237, 1486, 4021)

In [192]:
iriq['Type'] = 'Earthquake'
mexeq['Type'] = 'Earthquake'
calwf['Type'] = 'Fire'
hurir['Type'] = 'Hurricane'

In [195]:
res = pd.concat([iriq,mexeq,calwf,hurir])

In [200]:
#res.to_csv('C:/Users/gsent/Documents/Spring_2020/Info_440/crisisNLP/combined_weather_data.csv', index = False)